连接neo4j数据库

In [ ]:
from neo4j import GraphDatabase

#define the parameters, host, query and keywords
uri = "xxxxxx"
driver = GraphDatabase.driver(uri, auth=("neo4j", "xxxxxx"))
session = driver.session()

neo4j查询函数

In [6]:
def run_query(tx, query, cid, product, attribute,attribute_val):
    result_list=[]
    for record in tx.run(query, customerid=cid,productname=product, attribute=attribute,value=attribute_val):
        result_list.append(record[0])
    return result_list

定义neo4j查询语句

In [2]:
check_q = ("MATCH (c:Customer)-[r:HAS]->(p:Product) " 
    "WHERE c.customer_id = $customerid AND p.product_name = $productname " 
    "RETURN DISTINCT properties(r)")

check_c = ("MATCH (c:Customer) " 
    "WHERE c.customer_id = $customerid " 
    "RETURN DISTINCT c")

update_q = ("MATCH (c:Customer)-[r:HAS]->(p:Product) " 
    "WHERE c.customer_id = $customerid AND p.product_name = $productname "
    "and r.TYPE = $attribute "
    "SET r.VALUE = toInteger($value) "
    "RETURN DISTINCT properties(r)")

定义意图字典

In [3]:
intent_dict = {'check':check_q, 'login':check_c, 'update':update_q}

加载Spacy NLP库

In [4]:
import spacy
nlp = spacy.load('en_core_web_lg')

加载自定义分词

In [5]:

#list of key intent, product and attribute
product_list = ['deposit','loan']
attribute_list = ['pricing','balance']
intent_list = ['check','update']

tokens_products = nlp(' '.join(product for product in product_list))
tokens_intent = nlp(' '.join(intent for intent in intent_list))
tokens_attribute = nlp(' '.join(attribute for attribute in attribute_list))

意图提取

In [9]:

def intent_entity_attribute_extraction(nlp, sentence, tokens_intent, tokens_product, tokens_attribute):
    #please implement your sentence classification here to extract the intent
    tokens = nlp(sentence)
    #use the NER to extract the entity regarding product

    intent_score= 0
    product_score = 0
    attribute_score = 0
    final_intent = ''
    final_product = ''
    final_attribute = ''
    final_attribute_value = ''

    threshold = 0.8

    for token in tokens:
        for intent in tokens_intent:
            curr_intent_score = token.similarity(intent)
            if curr_intent_score > intent_score and curr_intent_score > threshold:
                intent_score = curr_intent_score
                final_intent = intent.text

        for product in tokens_product:
            curr_product_score = token.similarity(product)
            if curr_product_score > product_score and curr_product_score > threshold:
                product_score = curr_product_score
                final_product = product.text
                    
        for attribute in tokens_attribute:
            curr_attribute_score = token.similarity(attribute)
            if curr_attribute_score > attribute_score and curr_attribute_score > threshold:
                attribute_score = curr_attribute_score
                final_attribute = attribute.text

        if token.pos_ == 'NUM' and token.text.isdigit():
            final_attribute_value = token.text

    print('matching...')
    print(final_intent, final_product, final_attribute, final_attribute_value)
    return (final_intent, final_product, final_attribute, final_attribute_value)
